In [ ]:
!pip install PyPDF2
import PyPDF2
import re

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    # Replace all newlines with spaces
    text = text.replace('\n', ' ')
    # Use regex to add newlines after sentence endings
    text = re.sub(r'(?<=[.?!]) +', '\n', text)
    return text

# Install the python-docx library
!pip install python-docx # This line installs the necessary library

# Extracting text from Word documents using python-docx
from docx import Document

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text
        if not para.text.endswith("\n"):
            text += "\n"
    return text

extracted_text = extract_text_from_docx("Problem Statement.docx")
print(extracted_text)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove non-alphanumeric characters (except for spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize the text into words
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Join the words back into a string
    clean_text = ' '.join(filtered_words)

    return clean_text

# Example usage
text_from_pdf = extract_text_from_pdf('Problem Statement.pdf')
clean_text = preprocess_text(text_from_pdf)
print(clean_text)


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install rake_nltk
!python -m spacy download en_core_web_sm

from rake_nltk import Rake  # For keyword extraction
import spacy  # For dependency parsing
import PyPDF2

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    clean_text = ' '.join(filtered_words)
    return clean_text

# Function to extract user requirements
def extract_user_requirements(text):
    # Keyword Extraction using RAKE
    r = Rake()
    r.extract_keywords_from_text(text)
    keywords = r.get_ranked_phrases_with_scores()

    # Filter keywords related to user needs (e.g., "need," "require," "want")
    user_requirement_keywords = [phrase for score, phrase in keywords if any(word in phrase for word in ["need", "require", "want"])]

    # Dependency Parsing using spaCy
    doc = nlp(text)

    # Extract sentences containing user requirement keywords
    user_requirement_sentences = []
    for sent in doc.sents:
        if any(keyword in sent.text for keyword in user_requirement_keywords):
            user_requirement_sentences.append(sent.text)

    return user_requirement_sentences

# Upload file in Google Colab if needed
# from google.colab import files
# uploaded = files.upload()

# Example usage
pdf_path = 'PS2.pdf'  # Make sure the file exists
text_from_pdf = extract_text_from_pdf(pdf_path)
clean_text = preprocess_text(text_from_pdf)
user_requirements = extract_user_requirements(clean_text)

print(user_requirements)



In [ ]:
!pip install spacy

In [56]:
import torch
from transformers import pipeline

# Load Google's abstractive summarization model
summarization_pipeline = pipeline("summarization", model="google/pegasus-xsum")

def generate_requirement_statement(text):
    """
    Generate structured user requirement statements using abstractive summarization.
    """
    summarized_text = summarization_pipeline(text, max_length=70, min_length=10, do_sample=False)
    return summarized_text[0]['summary_text']

# Example usage
pdf_path = 'Problem Statement.pdf'  # Make sure the file exists
text_from_pdf = extract_text_from_pdf(pdf_path)
clean_text = preprocess_text(text_from_pdf)

# Generate structured requirement statement
requirement_statement = generate_requirement_statement(clean_text)
print("Generated Requirement Statement:", requirement_statement)



Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Generated Requirement Statement: Hospital management app allows patients to book appointments online doctors based availability app secure system storing patient records prescribe medicines digitally separate section staff Hospital management app allows patients to book appointments online doctors based availability app secure system storing patient records prescribe medicines digitally separate section staff Hospital management app allows patients to book appointments online doctors based availability app secure system storing patient records prescribe medicines
